In [2]:
import pandas as pd
import numpy as np

In [8]:
train = pd.read_csv("./data/7. train 이상치수정완료.csv",encoding='cp949',index_col=0)
train.head()
test1 = pd.read_csv("./data/6. test1.csv",index_col=0)
test2 = pd.read_csv("./data/6. test2.csv",index_col=0)

In [9]:
col = train.columns[7:55]
train_1 = train[col]
test_1 = test1[col]
test_2 = test2[col]

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_1)

train_Min = scaler.transform(train_1)
test1_Min = scaler.transform(test_1)
test2_Min = scaler.transform(test_2)
train_Min = pd.DataFrame(train_Min,columns = train_1.columns)

In [11]:
train[col] = train_Min
test1[col] = test1_Min
test2[col] = test2_Min

# 1. MDA

In [12]:
# 1-1 정규성 테스트(샤피로)
from scipy.stats import norm
from scipy import stats
from statsmodels.formula.api import ols
from scipy.stats import kstest

for j in col:
    a = stats.shapiro(train_Min[j])
    p = a.pvalue
    print(j,a)

매출액증가율 ShapiroResult(statistic=0.8382039070129395, pvalue=6.445972935894159e-44)
유형자산증가율 ShapiroResult(statistic=0.501219630241394, pvalue=0.0)
재고자산증가율 ShapiroResult(statistic=0.5221827030181885, pvalue=0.0)
비유동자산증가율 ShapiroResult(statistic=0.7331200838088989, pvalue=0.0)
유동자산증가율 ShapiroResult(statistic=0.8816322088241577, pvalue=3.204322573700737e-39)
자기자본증가율 ShapiroResult(statistic=0.8314107656478882, pvalue=1.5414283107572988e-44)
총자본증가율 ShapiroResult(statistic=0.8660170435905457, pvalue=4.744656470357398e-41)
순이익증가율 ShapiroResult(statistic=0.5167244672775269, pvalue=0.0)
매출액총이익률 ShapiroResult(statistic=0.9277034997940063, pvalue=2.2634414088783835e-32)
매출액순이익률 ShapiroResult(statistic=0.4788691997528076, pvalue=0.0)
자기자본순이익률 ShapiroResult(statistic=0.6980950832366943, pvalue=0.0)
총자본사업이익률 ShapiroResult(statistic=0.9500695466995239, pvalue=1.1103028499851191e-27)
총자본순이익률 ShapiroResult(statistic=0.8355908393859863, pvalue=3.6433760072445244e-44)
1주당매출액 ShapiroResult(statistic=0.446581

In [13]:
# 1-2 정규성 테스트(K-S)
for j in col:
    a = kstest(train_Min[j],'norm')
    p = a.pvalue
    print(j,a)


매출액증가율 KstestResult(statistic=0.5406899045861261, pvalue=0.0)
유형자산증가율 KstestResult(statistic=0.506022496636173, pvalue=0.0)
재고자산증가율 KstestResult(statistic=0.5, pvalue=0.0)
비유동자산증가율 KstestResult(statistic=0.5460829454049041, pvalue=0.0)
유동자산증가율 KstestResult(statistic=0.5522467939752654, pvalue=0.0)
자기자본증가율 KstestResult(statistic=0.578566142463263, pvalue=0.0)
총자본증가율 KstestResult(statistic=0.5866526747600764, pvalue=0.0)
순이익증가율 KstestResult(statistic=0.5, pvalue=0.0)
매출액총이익률 KstestResult(statistic=0.5170644630842823, pvalue=0.0)
매출액순이익률 KstestResult(statistic=0.6925703515300512, pvalue=0.0)
자기자본순이익률 KstestResult(statistic=0.6362943004377452, pvalue=0.0)
총자본사업이익률 KstestResult(statistic=0.5218984214196939, pvalue=0.0)
총자본순이익률 KstestResult(statistic=0.5491686968923792, pvalue=0.0)
1주당매출액 KstestResult(statistic=0.5, pvalue=0.0)
1주당순이익 KstestResult(statistic=0.5418746917070745, pvalue=0.0)
유동비율 KstestResult(statistic=0.5009265578228983, pvalue=0.0)
당좌비율 KstestResult(statistic=0.50020581798814

In [14]:
train_Min["label"] = train["label"]

#### p-value가 0 이는 데이터개수가 많아서 p-value 자체가 너무작아 계산이 불가능하다고 판단. 중심극한 정리에 의해서 정규성이 있다고 가정하고 진행

In [15]:
# 부도기업과 정상기업의 피처별 등분산비교 (정규성 가정으로 bartlett)
Bad = train_Min[train_Min['label']== 1] #Existing Customer
Good = train_Min[train_Min['label']== 0] #Attrited Customer

c = []
for i in train_Min:
    # lresult = stats.levene(close[i], normal[i])
    lresult = stats.bartlett(Bad[i], Good[i])
    c.append([i,lresult[-1]])

c= pd.DataFrame(c)
c.columns=["피처값",'F-test']
c

c:\Users\OPERATOR-70\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_morestats.py:2507: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\OPERATOR-70\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_morestats.py:2507: RuntimeWarning: invalid value encountered in double_scalars
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)


,피처값,F-test
0,매출액증가율,1.417322e-06
1,유형자산증가율,9.425294e-10
2,재고자산증가율,1.156878e-02
3,비유동자산증가율,3.089934e-04
4,유동자산증가율,1.768075e-02
5,자기자본증가율,1.764859e-12
6,총자본증가율,2.079932e-01
7,순이익증가율,7.329558e-19
8,매출액총이익률,1.449988e-04
9,매출액순이익률,1.157832e-32


In [16]:
# F-test 결과 0.05 이상이면 homo 0.05 이하이면 hetero
c["분산"] =''
c["T-test"] =""
for i in c.index:
    if c.loc[i,"F-test"]>=0.05:
        c.loc[i,"분산"] = "homo"
    else:
        c.loc[i,"분산"] = "hetero"
c

,피처값,F-test,분산,T-test
0,매출액증가율,1.417322e-06,hetero,
1,유형자산증가율,9.425294e-10,hetero,
2,재고자산증가율,1.156878e-02,hetero,
3,비유동자산증가율,3.089934e-04,hetero,
4,유동자산증가율,1.768075e-02,hetero,
5,자기자본증가율,1.764859e-12,hetero,
6,총자본증가율,2.079932e-01,homo,
7,순이익증가율,7.329558e-19,hetero,
8,매출액총이익률,1.449988e-04,hetero,
9,매출액순이익률,1.157832e-32,hetero,


In [17]:
c[c["분산"]=='homo']

,피처값,F-test,분산,T-test
6,총자본증가율,0.207993,homo,
15,유동비율,0.768681,homo,
16,당좌비율,0.723862,homo,
17,현금비율,0.600805,homo,
26,자기자본회전률,0.630710,homo,
36,순운전자본회전률,0.867793,homo,
39,당좌자산회전률,0.950182,homo,


In [18]:
# homo 인 feature 는 student t-test, hetero이면 Welchs T-Test 진행
c["분산"] =''
c["T-test"] =""
for i in c.index:
    if c.loc[i,"F-test"]>=0.05:
        c.loc[i,"분산"] = "homo"
        result = stats.ttest_ind(Bad[c.loc[i,"피처값"]], Good[c.loc[i,"피처값"]], equal_var=True)       ## equal_var = True Student T-test
        c.loc[i,"T-test"] = result[-1]
    else:
        c.loc[i,"분산"] = "hetero"
        result = stats.ttest_ind(Bad[c.loc[i,"피처값"]], Good[c.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test
        c.loc[i,"T-test"] = result[-1]
c

C:\Users\OPERATOR-70\AppData\Local\Temp\ipykernel_10780\43181667.py:11: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  result = stats.ttest_ind(Bad[c.loc[i,"피처값"]], Good[c.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test


,피처값,F-test,분산,T-test
0,매출액증가율,1.417322e-06,hetero,0.001358
1,유형자산증가율,9.425294e-10,hetero,0.558464
2,재고자산증가율,1.156878e-02,hetero,0.015938
3,비유동자산증가율,3.089934e-04,hetero,0.002837
4,유동자산증가율,1.768075e-02,hetero,0.0
5,자기자본증가율,1.764859e-12,hetero,0.000065
6,총자본증가율,2.079932e-01,homo,0.0
7,순이익증가율,7.329558e-19,hetero,0.000022
8,매출액총이익률,1.449988e-04,hetero,0.074929
9,매출액순이익률,1.157832e-32,hetero,0.000003


In [19]:
# 0.05 이상이면 유의미하지 않으므로 0.05 이하인 것만 추출
d = c[c["T-test"]<= 0.05]
d.sort_values('T-test',ascending=False)["피처값"].unique()
# d.sort_values('T-test',ascending=False)["피처값"]

array(['유동자산구성비율', '자본분배율', '이자보상배율이자비용', '매출채권회전기간', '자기자본회전률',
       '재고자산증가율', '현금비율', '당좌비율', '부채비율', '비유동비율', '매입채무회전기간', '당좌자산회전률',
       '유동비율', '비유동자산증가율', '1주당매출액', '설비투자효율', '매출액증가율', '자기자본증가율',
       '부가가치율', '순이익증가율', '자기자본구성비율', '총자본투자효율', '매입채무회전률', '매출액순이익률',
       '자기자본순이익률', '총자본회전률', '1주당순이익', '유동자산증가율', '순운전자본비율', '순운전자본회전률',
       '부가가치', '총자본순이익률', '총자본사업이익률', '비유동자산회전률', '기계투자효율', '자기자본배율',
       '총자본증가율', 'label'], dtype=object)

In [20]:
d.sort_values('T-test',ascending=False).dropna()

,피처값,F-test,분산,T-test
22,유동자산구성비율,3.318390e-02,hetero,0.03795
46,자본분배율,3.334576e-11,hetero,0.032797
20,이자보상배율이자비용,1.537723e-03,hetero,0.031688
33,매출채권회전기간,5.345712e-13,hetero,0.030651
26,자기자본회전률,6.307105e-01,homo,0.019695
2,재고자산증가율,1.156878e-02,hetero,0.015938
17,현금비율,6.008051e-01,homo,0.013937
16,당좌비율,7.238616e-01,homo,0.013597
19,부채비율,3.098096e-32,hetero,0.008211
21,비유동비율,1.651442e-35,hetero,0.007159


In [34]:
d.sort_values('T-test',ascending=False).dropna()["피처값"].tolist()

['유동자산구성비율',
 '자본분배율',
 '이자보상배율이자비용',
 '매출채권회전기간',
 '자기자본회전률',
 '재고자산증가율',
 '현금비율',
 '당좌비율',
 '부채비율',
 '비유동비율',
 '매입채무회전기간',
 '당좌자산회전률',
 '유동비율',
 '비유동자산증가율',
 '1주당매출액',
 '설비투자효율',
 '매출액증가율',
 '자기자본증가율',
 '부가가치율',
 '순이익증가율',
 '자기자본구성비율',
 '총자본투자효율',
 '매입채무회전률',
 '매출액순이익률',
 '자기자본순이익률',
 '총자본회전률',
 '1주당순이익',
 '유동자산증가율',
 '순운전자본비율',
 '순운전자본회전률',
 '부가가치',
 '총자본순이익률',
 '총자본사업이익률',
 '비유동자산회전률',
 '기계투자효율',
 '자기자본배율',
 '총자본증가율']

In [21]:
# t-test가 가장 작은순으로 의미 있으므로 가장작은 순위로 10개 피처 선정
fea = ['자기자본구성비율', '매출액순이익률', '순운전자본비율',
       '순운전자본회전률', '총자본투자효율', '자기자본순이익률', '총자본증가율', '총자본사업이익률', '기계투자효율',
       '1주당순이익', '총자본순이익률', '자기자본배율']
mda_feature = train[fea]
mda_feature 

,자기자본구성비율,매출액순이익률,순운전자본비율,순운전자본회전률,총자본투자효율,자기자본순이익률,총자본증가율,총자본사업이익률,기계투자효율,1주당순이익,총자본순이익률,자기자본배율
0,0.518287,0.704184,0.831249,0.151079,0.270149,0.585951,0.990141,0.210883,0.011194,0.240763,0.311800,0.125460
1,0.872098,0.812082,0.744764,0.151079,0.330911,0.692925,0.430612,0.358916,0.011387,0.246979,0.442283,0.128066
2,0.836618,0.810149,0.567651,0.201439,0.329145,0.691296,0.460540,0.340383,0.011278,0.246877,0.440362,0.128068
3,0.855563,0.806766,0.603102,0.730216,0.463941,0.691180,0.357728,0.338600,0.012579,0.247016,0.431876,0.161256
4,0.896846,0.736971,0.853191,0.607914,0.283119,0.533085,0.450430,0.304503,0.011342,0.224059,0.156100,0.145324
...,...,...,...,...,...,...,...,...,...,...,...,...
2365,0.375712,0.775811,0.375384,0.312950,0.348827,0.591847,0.397465,0.246525,0.013592,0.219599,0.345501,0.175988
2366,0.080158,0.691517,0.186977,0.251799,0.218067,0.217128,0.288617,0.096828,0.009072,0.174297,0.133445,0.127505
2367,0.323368,0.793387,0.300979,0.312950,0.273530,0.636646,0.435391,0.271831,0.011190,0.188202,0.393852,0.380030
2368,0.662615,0.816787,0.695661,0.841727,0.339793,0.736366,0.417283,0.458833,0.013999,0.271459,0.505283,0.238572


In [22]:
def vif(data):
    import pandas as pd
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    # VIF 출력을 위한 데이터 프레임 형성
    vif = pd.DataFrame()

    # VIF 값과 각 Feature 이름에 대해 설정
    vif["VIF Factor"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]
    vif["features"] = data.columns 
                    
    # VIF 값이 높은 순으로 정렬
    vif = vif.sort_values(by="VIF Factor", ascending=False)
    vif = vif.reset_index().drop(columns='index')
    
    return vif

vif(mda_feature)

,VIF Factor,features
0,146.756708,총자본순이익률
1,116.289703,자기자본순이익률
2,70.476141,매출액순이익률
3,42.402629,총자본사업이익률
4,42.022691,총자본투자효율
5,19.577351,총자본증가율
6,18.337503,순운전자본비율
7,16.752791,자기자본구성비율
8,15.426970,1주당순이익
9,6.614164,순운전자본회전률


# 3. Lasso

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [55]:
# 12개 피처로 조정 C 값 조정
lasso = SelectFromModel(LogisticRegression(C=0.6, penalty='l1',solver='liblinear'))
lasso.fit(feature, target)
lasso_support = lasso.get_support()
lasso_feature = feature.loc[:,lasso_support].columns.tolist()

In [56]:
lasso_feature

['유동자산증가율',
 '총자본증가율',
 '총자본사업이익률',
 '총자본순이익률',
 '순운전자본비율',
 '부채비율',
 '자기자본배율',
 '매출채권회전률',
 '매입채무회전기간',
 '매출채권회전기간',
 '순운전자본회전률',
 '당좌자산회전률']

In [ ]:
## 성장성'유동자산증가율','총자본증가율',
## 수익성  '자기자본순이익률', '총자본순이익률',
## 생산성 '자본분배율',  '부가가치'
## 안정성 '순운전자본비율', '자기자본배율'
## 활동성 "매입채무회전률",'매출채권회전기간'

In [1]:
fea = [
'유동자산증가율','총자본증가율',
  '자기자본순이익률', '총자본순이익률',
'자본분배율',  '부가가치',
'순운전자본비율', '자기자본배율',
 "매입채무회전률",'매출채권회전기간',"label"]

lasso_feature = train[fea]
lasso_feature 

NameError: name 'train' is not defined